In [5]:
import pandas as pd
from sklearn.preprocessing import PowerTransformer
import numpy as np

# Cargar datos
df = pd.read_csv("data_proyecto2-1.csv", parse_dates=["Fecha"], dayfirst=True)

# Separar variables x1 y x2
x1_df = df[df['nombre'] == 'x1'].copy()
x2_df = df[df['nombre'] == 'x2'].copy()

In [7]:
pt = PowerTransformer(method='yeo-johnson')

# Aplicar transformación a x1
x1_df['data_trans'] = pt.fit_transform(x1_df[['data']])

# Aplicar transformación a x2
x2_df['data_trans'] = pt.fit_transform(x2_df[['data']])


In [9]:
def clip_outliers_iqr(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series.clip(lower, upper)

x1_df['data_clean'] = clip_outliers_iqr(x1_df['data_trans'])
x2_df['data_clean'] = clip_outliers_iqr(x2_df['data_trans'])

In [11]:
# Puedes usar interpolación o media
x1_df['data_clean'].fillna(x1_df['data_clean'].mean(), inplace=True)
x2_df['data_clean'].fillna(x2_df['data_clean'].mean(), inplace=True)

C:\Users\joseg\AppData\Local\Temp\ipykernel_38672\3845742753.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x1_df['data_clean'].fillna(x1_df['data_clean'].mean(), inplace=True)
C:\Users\joseg\AppData\Local\Temp\ipykernel_38672\3845742753.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [13]:
# Definir ancho de ventana (ej. 30 días)
window = 30

# Rolling stats para x1
x1_df['mean_rolling'] = x1_df['data_clean'].rolling(window).mean()
x1_df['std_rolling'] = x1_df['data_clean'].rolling(window).std()
x1_df['trend_rolling'] = x1_df['data_clean'].rolling(window).apply(lambda x: x.iloc[-1] - x.iloc[0], raw=False)

# Rolling stats para x2
x2_df['mean_rolling'] = x2_df['data_clean'].rolling(window).mean()
x2_df['std_rolling'] = x2_df['data_clean'].rolling(window).std()
x2_df['trend_rolling'] = x2_df['data_clean'].rolling(window).apply(lambda x: x.iloc[-1] - x.iloc[0], raw=False)


In [15]:
x1_df

,nombre,Fecha,data,data_trans,data_clean,mean_rolling,std_rolling,trend_rolling
0,x1,2006-01-01,6.05,-0.620684,-0.620684,NaN,NaN,NaN
1,x1,2006-01-02,4.72,-0.674254,-0.674254,NaN,NaN,NaN
2,x1,2006-01-03,4.46,-0.685094,-0.685094,NaN,NaN,NaN
3,x1,2006-01-04,34.74,0.244439,0.244439,NaN,NaN,NaN
4,x1,2006-01-05,1.91,-0.800454,-0.800454,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3282,x1,2014-12-27,13.16,-0.368699,-0.368699,-0.231833,0.299144,-0.214172
3283,x1,2014-12-28,8.50,-0.528469,-0.528469,-0.244297,0.303570,-0.596531
3284,x1,2014-12-29,11.03,-0.439667,-0.439667,-0.261222,0.299683,-0.250224
3285,x1,2014-12-30,21.33,-0.118675,-0.118675,-0.258863,0.300545,-0.100677


In [17]:
x2_df

,nombre,Fecha,data,data_trans,data_clean,mean_rolling,std_rolling,trend_rolling
3287,x2,2006-01-01,9.22,0.050542,0.050542,NaN,NaN,NaN
3288,x2,2006-01-02,-13.57,-2.248973,-1.627926,NaN,NaN,NaN
3289,x2,2006-01-03,5.95,-0.139089,-0.139089,NaN,NaN,NaN
3290,x2,2006-01-04,3.26,-0.308375,-0.308375,NaN,NaN,NaN
3291,x2,2006-01-05,7.34,-0.056775,-0.056775,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6566,x2,2014-12-27,-13.30,-2.209939,-1.627926,-0.310186,0.794404,-1.341877
6567,x2,2014-12-28,16.20,0.421316,0.421316,-0.286607,0.805564,0.497968
6568,x2,2014-12-29,5.10,-0.190953,-0.190953,-0.290417,0.804807,-0.441469
6569,x2,2014-12-30,4.90,-0.203351,-0.203351,-0.305546,0.798529,1.424575
